In [2]:
import numpy as np
import datasets

datasets.downloads = '/users/gregc/Downloads/'

In [2]:
trX, teX, trY, teY = datasets.mnist()

In [121]:
decode = lambda p: np.argmax(p, axis = 1) 
accuracy = lambda p, Y: np.mean(decode(p) == decode(Y))

In [4]:
import tensorflow as tf

def dense(output_dim, x):
    shape = (x.get_shape()[1].value, output_dim)
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return tf.matmul(x, w) + b  
    
def fit(sess, x, y, X, Y, optimizer, epochs):
    sess.run(tf.initialize_all_variables())
    N, D = X.shape

    for _ in range(epochs):
        for start in range(0, N - 32, 32):
            end = start + 32
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

In [5]:
x = tf.placeholder(tf.float32, (None, trX.shape[1]))
y = tf.placeholder(tf.float32)
h = tf.nn.relu(dense(625, x))  
model = dense(trY.shape[1], h)
loss = tf.nn.softmax_cross_entropy_with_logits(model, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    fit(sess, x, y, trX, trY, optimizer, epochs = 1)
    print accuracy(model.eval({x: teX}), teY)

0.9531


In [73]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(output_dim = 625, input_dim = trX.shape[1], activation = 'relu'))
model.add(Dense(output_dim = trY.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = 1)
print accuracy(model.predict_proba(teX, verbose = 0), teY)

Epoch 1/1
60000/60000 [==============================] - 32s - loss: 0.1944    
0.9691


In [7]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)

In [12]:
# http://iamtrask.github.io/2015/07/12/basic-python-network/

X, y = datasets.htwt()
y = np.array([y]).T

# X = np.array([ [0, 0, 1],[0, 1, 1],[1, 0, 1],[1, 1, 1] ])
# y = np.array([[0, 1, 1, 0]]).T

epochs = 10000
learning_rate = 0.00001

N, D = X.shape

W0 = 2 * np.random.random((D, N)) - 1
W1 = 2 * np.random.random((N, 1)) - 1

for j in xrange(epochs):
    l1 = 1 / (1 + np.exp(-(np.dot(X, W0))))
    l2 = 1 / (1 + np.exp(-(np.dot(l1, W1))))
    l2_delta = (y - l2) * (l2 * (1 - l2))
    l1_delta = l2_delta.dot(W1.T) * (l1 * (1 - l1))
    W1 += l1.T.dot(l2_delta) * learning_rate
    W0 += X.T.dot(l1_delta) * learning_rate
    
print np.mean((l2.T[0] > 0.5) * 1 == y)

0.547891156463


In [104]:
import numpy as np

class MyDense:
    def __init__(self, output_dim, X):
        self.X = X
        self.W = np.random.normal(0, 0.05, (output_dim, X.shape[1]))
        self.b = np.zeros((output_dim, 1))
        self.shape = X.shape
        
    def forward(self):
        self.eta = self.W.dot(X.T) + np.tile(self.b, (1, self.X.shape[0]))

    def back(self):
        pass

    def update(self):
        pass

X = np.array([[1, 1], [0, 2], [4, 3]])
        
dense = MyDense(100, X)
dense.forward()
# dense.eta